In [4]:
from field import F, P, ZERO, root

def get(xs, i, default_val):
    return xs[i] if i < len(xs) else default_val

class Polynomial:
    def __init__(self, cs, zero = ZERO):
        # Remove trailing 0s
        cs = [c for c in cs]
        while len(cs) > 0 and cs[-1] == zero:
            cs.pop()

        if len(cs) == 0:
            cs.push(zero)

        self.cs = [c for c in cs]
        self.z = zero

    def degree(self):
        l = len(self.cs)
        assert l > 0
        return l - 1

    def __neg__( self ):
        return Polynomial([-c for c in self.cs], self.z)
    
    def __add__(self, r):
        ls = self.cs
        rs = r.cs
        cs = [self.z] * max(len(ls), len(rs))
        for i in range(len(cs)):
            a = get(ls, i, self.z)
            b = get(rs, i, self.z)
            cs[i] = a + b
        return Polynomial(cs, self.z)

    def __sub__(self, r):
        return self.__add__(-r)

    def __mul__(self, r):
        m = len(self.cs)
        n = len(r.cs)
        ls = self.cs
        rs = r.cs

        # degree = len(cs) - 1
        # degree(self) + degree(r) = len(cs) - 1
        # len(cs) = degree(self) + degree(r) + 1
        #         = len(self.cs) - 1 + len(r.cs) - 1 + 1
        cs = [self.z] * (m + n - 1)
        for i in range(m):
            for j in range(n):
                cs[i + j] += ls[i] * rs[j]

        return Polynomial(cs, self.z)

    def __eq__(self, r):
        return self.cs == r.cs

    def __neq__(self, r):
        return self.cs != r.cs

    def __str__(self):
        return str(self.cs)

    def __repr__(self):
        return str(self.cs)

p = Polynomial([F(1), F(2)])
r = Polynomial([F(1), F(2), F(3)])
z = p * r
z

[1, 4, 7, 6]

In [ ]:
# Polynomials
def poly_eval(p, x):
    y = 0
    xi = 1
    for c in p:
        y += c * xi
        xi *= x
    return y

# 0 <= P(x) <= 9 integer 
# for all 1 <= x <= 1,000,000

# C(x) = x*(x-1)*...*(x-9)
# C(x) = 0 if 0 <= x <= 9
def c(x):
    v = 1
    for i in range(10):
        v *= (x - i)
    return v

# Z(x) = (x-1)*(x-2)*...*(x-1e6)
def z(x, n):
    v = 1
    for i in range(1, n + 1):
        v *= (x - i)
    return v

# C(P(x)) = 0 for 1 <= x <= 1,000,000
# C(P(x)) = Z(x)D(x)

# Polynomial commitment
# 1. Merkle tree of P(x) and D(x) at 1,000,000,000 points
# 2. Verifier randomly selects 16 values between 1 and 1,000,000,000
#    and asks the prover to provide the Merkle branches of P(x) and D(x)
# 3. Verifier checks
#    - Merkle proofs
#    - C(P(x)) = Z(x)D(x), P(x) and D(x) are provided in the Merkle proof

# Bivariate polynomial
# f(x) = polynomial with degree < 1,000,000
# Find g(x,y) such that g(x, x^1000) = f(x)

# 1. Polynomial commintment on g(x, y)
# for {(x, y^1000) for 1 <= x <= N and 1 <= y <= N}
# N = 1,000,000,000

# 2. Verifier picks randomly picks a few rows and columns
#    and for each row and column, asks for a few sample of points
#    one point in the sample is on the diagnol (x, x^1000)

# 3. Prover replies with does points and Merkle proofs

# TODO: why use bivariate polynomials?
# 4. - Verifier checks Merkle proofs
#    - Check polynomial corresponds to a low degree polynomial.
#      Degree less than number of samples requested.